In [ ]:
from src.preprocessing import clean_inequality_data
from src.split_data import split_data

df = clean_inequality_data()

X_train, X_test, y_train, y_test = split_data(df, "gini")

X_train.shape, X_test.shape, y_train.shape, y_test.shape